# Traducción de protocolos
--------

Herramientas para traducir los protocolos clínicos y de estructuras.

### Modo de trabajo
- Deifinir un diccionario con la correspondencia entre el nombre actual de la estructura y el que se quiere introducir
- Exportar de ARIA los protocolos clínicos y de contorneo
- Aplicar una función de traducción, que comprueba si existe en el correspondiente protocolo cada una de las estructuras del diccionario, a cada uno de los protocolos exportados
- Guardar el protocolo corregido en un archivo diferente para no perder los originales.
- Actualizar en ARIA los protocolos, eliminando el anterior e importando el modificado

Importación de módulos

In [31]:
import pandas as pd
import numpy as np
from glob import glob
from pathlib import Path
from tqdm.notebook import tqdm

Ir al directorio de trabajo, directorio base del repositorio

In [2]:
%cd ..

/Users/maqui/Documents/devel/rtpros


Eliminar los duplicados del Diccionario original. Esta celda solo es necesaria si el Diccionario aún no se ha acondicionado

In [3]:
dodf = pd.read_csv('tools/DiccionarioOriginal.csv', dtype={'est': str, 'strt' : str, 'code' : object, 'ncode' : object})
dodf.drop_duplicates(inplace=True)
dodf.to_csv('tools/Diccionario.csv', index=False)

Leer el diccionario

In [4]:
ddf = pd.read_csv('tools/Diccionario.csv')
ddf

,est,strt,code,ncode
0,Cabeza humeral I,Humerus_L,25929.0,NaN
1,Cabeza humeral D,Humerus_R,25927.0,NaN
2,Pulmón Izqdo,Lung_L,7310.0,NaN
3,Pulmón Dcho,Lung_R,7309.0,NaN
4,Médula,SpinalCord,7647.0,NaN
...,...,...,...,...
104,Hígado Sano,Liver-CTV,NaN,NaN
105,Duodeno,Duodenum,NaN,7206
106,Ambos riñones,Kidneys,NaN,264815
107,Ambos pulmones,Lungs,NaN,68877


#### Función de traducción
Leer un protocolo clínico exportado en formato `xml`.

    Por como está formado el archivo xml, al leerlo en pandas se crea una serie con el tipo de archivo xml en la cabecera del DataFrame y el resto de datos en un único elemento de una serie.

    Es útil leerlo en pandas para emplear sus herramientas de sustitución de texto.

Aplicar la sustitución sobre el `ID` y el `Name` de la estructura. Esta versión no realiza comprobación del código de la estructura 


In [11]:
prdf = pd.read_csv('protocolos/clinicos/MamaDchaCads15Fx.xml')

In [12]:
for index, row in ddf.iterrows():
    try:
        match = 'ID="' + row.est + '" Name="' + row.est + '"'
        repl = 'ID="' + row.strt + '" Name="' + row.strt + '"'
        prdf.iloc[0] = prdf.iloc[0].str.replace(match, repl)
    except TypeError:
        pass

In [14]:
with open('protocolos/clinicos/modificados/MamaDchaCads15Fx.xml', 'w') as f:
    f.write(prdf.to_string(index=False).lstrip())

In [21]:
pfs = glob('protocolos/clinicos/*.xml')

In [24]:
p = Path(pfs[0])

IndexError: list index out of range

In [28]:
p.parents[0].joinpath('modificados').joinpath(p.name)

PosixPath('protocolos/clinicos/modificados/MamaDchaCads15Fx.xml')

In [33]:
def traducirProtocoloClinico(pf=None, ddf=None):
    path = Path(pf)
    prdf = pd.read_csv(path)

    for index, row in ddf.iterrows():
        try:
            match = 'ID="' + row.est + '" Name="' + row.est + '"'
            repl = 'ID="' + row.strt + '" Name="' + row.strt + '"'
            prdf.iloc[0] = prdf.iloc[0].str.replace(match, repl)
            match = 'Item ID="' + row.est + '"'
            repl = 'Item ID="' + row.strt + '"'
            prdf.iloc[0] = prdf.iloc[0].str.replace(match, repl)
        except TypeError:
            pass

    with open(path.parents[0].joinpath('modificados').joinpath(path.name), 'w') as f:
        f.write(prdf.to_string(index=False).lstrip())

In [35]:
for pf in tqdm(pfs):
    traducirProtocoloClinico(pf, ddf)

  0%|          | 0/1 [00:00<?, ?it/s]